# Préambule

## Introduction & Contexte

Ce projet a été réalisé par 3 étudiants de Master 1 à l'ENSAE, dans le cadre du cours "Python for Data Science". L'idée de ce projet vient d'une intuition que nous avons eu selon laquelle il était possible de prédire le succès d'un jeu vidéo auprès des utilisateurs à partir de certaines de ses caractéristiques. L'objectif principal de ce travail est donc de tenter de vérifier ou infirmer cette intuition à l'aide de techniques statistiques de traitement de données. 

De plus, nous voulions y incorporer une partie plus originale (traitement d'images...)

# Importation des modules 

## Modules de webscrapping

On installe les packages utiles au Webscrapping des données sur les jeux

In [ ]:
pip install requests_html

In [2]:
# On importe les packages

import requests 
import urllib
import bs4
from requests_html import HTMLSession



## Importation des modules de Visualisation & Modélisation des données

In [3]:
#Là on importe les autres packages
import numpy as np
import pandas as pd

# I/ Récupération des données

## I/A Webscrapping des titres des jeux vidéos sur Wikipédia

A partir du site Wikipédia, nous allons récupérer les titres de tous les jeux vidéos sortis depuis l'année 2000. Par exemple, depuis l'URL : https://en.wikipedia.org/wiki/Category:2023_video_games nous avons accès à la liste de la quasi-totalité des jeux vidéos sortis en 2023. Nous allons donc modifier l'URL pour chaque année et constituer une liste (Liste_VG)

In [ ]:

from tqdm import tqdm
Liste_VG = []

d = {17: 11, 18 : 9, 19 : 9, 20: 12, 21: 11, 22 : 7}
for i in range(17, 23) :
    
    
    response = requests.get(url="https://en.wikipedia.org/wiki/20"+str(i)+"_in_video_games")
    soup = bs4.BeautifulSoup(response.content, 'html.parser')
    tableau_participants = soup.findAll('table', {'class' : 'wikitable'})
    for j in range(d[i], d[i]+4) :
        rows = tableau_participants[j].find_all('tr')
        first = True
        for row in rows :
            if first :
                first = False
            else:
                title = row.find('i')
                if title != None :
                    Liste_VG.append((row.find('i').text))
                    
print(Liste_VG)


## I/B Récupération des notes des jeux

L'objectif de cette partie est d'extraire du site "Métacritic", un site spécialisé qui répertorie les avis de professionnels et d'utilisateurs sur les nouvelles sorties jeux vidéos. Pour chaque jeu dans la base de données du site, une note sur 100 est attribuée (moyenne des reviews de sites spécialisés jeux vidéos) et une note sur 10 attribuée par les utilisateurs du site. Nous allons ici récupérer pour chaque jeu ses notes Métacritic qui deviendront les variables que nous tenteront de prédire par la suite. 

On remarque que l'on peut atteindre la review du jeu à partir de l'URL, il suffit d'indiquer le titre du jeu dans l'URL, en faisant attention aux espaces. 

In [ ]:
#URL du site : https://www.metacritic.com/game/
#URL d'une recherche : https://www.metacritic.com/search/gran%20theft%20auto/
#URL d'une review : https://www.metacritic.com/game/grand-theft-auto-v/
#URL d'une review d'un autre site spécialisé : https://www.ign.com/games/grand-theft-auto-v

#Base URL de toutes les reviews
URL = "https://www.metacritic.com/game/"
URLbis = "https://www.metacritic.com/game/grand-theft-auto-v/"
user_agent = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}
site = requests.get(URLbis, user_agent)

print(site.status_code)

In [ ]:
session = HTMLSession()


In [ ]:
DOM_Html = session.get(URLbis)
#chemin_note  = DOM_Html.html.find('body > div > div > div.c-layoutDefault > div.c-layoutDefault_page > div.c-pageProductGame \
                                #> div > div.c-productHero > div.c-productHero_container > div.c-productHero_player-scoreInfo \
                                #> div.c-productHero_score-container > div.c-productHero_scoreInfo > div > div.c-productScoreInfo \
                                #> div.c-productScoreInfo_scoreContent > div.c-productScoreInfo_scoreNumber > div.c-siteReviewScore_background \
                                #> div ', first = True)

#On accède sur chaque page aux notes que l'on souhaite via le chemin et on extrait la note d'une façon un peu wtf
#Sur le site on peut aussi extraire des trucs genre la date, les plateformes, le nombre de critiques et de notes, on a les comments de chacun, le nombre d'avis positifs/neutres/négatifs...

chemin_metascore = DOM_Html.html.find('#__layout > div > div.c-layoutDefault_page > div.c-pageProductGame > div:nth-child(1) > div > div \
                                  > div.c-productHero_player-scoreInfo.u-grid.g-grid-container > \
                                  div.c-productHero_score-container.u-flexbox.u-flexbox-column.g-bg-white > \
                                  div.c-productHero_scoreInfo.g-inner-spacing-top-medium.g-outer-spacing-bottom-medium.g-outer-spacing-top-medium \
                                  > div:nth-child(1) > div > div.c-productScoreInfo_scoreContent.u-flexbox.u-flexbox-alignCenter.u-flexbox-justifyFlexEnd.g-width-100.u-flexbox-nowrap > \
                                  div.c-productScoreInfo_scoreNumber.u-float-right > div > div', first = True)

chemin_userscore = DOM_Html.html.find('#__layout > div > div.c-layoutDefault_page > div.c-pageProductGame > div:nth-child(1) > div > div > \
                                      div.c-productHero_player-scoreInfo.u-grid.g-grid-container > div.c-productHero_score-container.u-flexbox.u-flexbox-column.g-bg-white > \
                                      div.c-productHero_scoreInfo.g-inner-spacing-top-medium.g-outer-spacing-bottom-medium.g-outer-spacing-top-medium > \
                                      div.c-productScoreInfo.u-clearfix > div.c-productScoreInfo_scoreContent.u-flexbox.u-flexbox-alignCenter.u-flexbox-justifyFlexEnd.g-width-100.u-flexbox-nowrap > \
                                      div.c-productScoreInfo_scoreNumber.u-float-right > div > div' , first = True)

#On reformate les titres afin d'avoir simplement les notes

metacritic_rating_inter = chemin_metascore.attrs['title']
metacritic_rating = metacritic_rating_inter.split(' ')[1]
users_ratings_inter = chemin_userscore.attrs['title']
users_ratings = float(users_ratings_inter.split(' ')[2])


#info_jeu = chemin_note.text
#info_jeu_split = info_jeu.split(' ')
#metacritic_rating = info_jeu_split[0][0:2]
#users_ratings = info_jeu_split[9][8:11]
#nb_ratings = int(info_jeu_split[6].replace(',', ''))

In [ ]:
#print(chemin_note.attrs['title'])
print(metacritic_rating, type(users_ratings))
#__layout > div > div.c-layoutDefault_page > div.c-pageProductGame > div:nth-child(1) > div > div > div.c-productHero_player-scoreInfo.u-grid.g-grid-container > div.c-productHero_score-container.u-flexbox.u-flexbox-column.g-bg-white > div.c-productHero_scoreInfo.g-inner-spacing-top-medium.g-outer-spacing-bottom-medium.g-outer-spacing-top-medium > div:nth-child(1) > div > div.c-productScoreInfo_scoreContent.u-flexbox.u-flexbox-alignCenter.u-flexbox-justifyFlexEnd.g-width-100.u-flexbox-nowrap > div.c-productScoreInfo_scoreNumber.u-float-right > div > div > span
#__layout > div > div.c-layoutDefault_page > div.c-pageProductGame > div:nth-child(1) > div > div > div.c-productHero_player-scoreInfo.u-grid.g-grid-container > div.c-productHero_score-container.u-flexbox.u-flexbox-column.g-bg-white > div.c-productHero_scoreInfo.g-inner-spacing-top-medium.g-outer-spacing-bottom-medium.g-outer-spacing-top-medium > div:nth-child(1) > div > div.c-productScoreInfo_scoreContent.u-flexbox.u-flexbox-alignCenter.u-flexbox-justifyFlexEnd.g-width-100.u-flexbox-nowrap > div.c-productScoreInfo_scoreNumber.u-float-right > div > div > span
#print(nb_ratings)

In [ ]:
DOM_Html1 = session.get('https://www.metacritic.com/game/assassins-creed-2007/')
DOM_Html2 = session.get('https://www.metacritic.com/game/red-dead-redemption-2/')
jeu1 = DOM_Html1.html.find('body > div > div > div.c-layoutDefault > div.c-layoutDefault_page > div.c-pageProductGame > div > div.c-productHero > div.c-productHero_container > div.c-productHero_player-scoreInfo > div.c-productHero_score-container > div.c-productHero_scoreInfo > div > div.c-productScoreInfo > div.c-productScoreInfo_scoreContent > div.c-productScoreInfo_scoreNumber > div.c-siteReviewScore_background > div', first = True).text
Jeu1 = jeu1.split(' ')
metacritic_rating = Jeu1[0][0:2]
users_ratings = Jeu1[9][8:11]
#print(Jeu1[10])